# Bebording
Set aan diverse analyses die fouten in het NDW bordenbestand opsporen.

In [1]:
import psycopg2
try:
    postgres_pw = TokenLibrary.getSecret("redacted", "redacted", "redacted")
    conn = psycopg2.connect(dbname="rdt_dev",user="redacted",host="redacted",password=postgres_pw)
    conn.autocommit = True
    cur=conn.cursor()
    print("Database connected")
except:
    print("I am unable to connect to the database")

## Tekst-waarde
De kolom 'black-code' uit het NDW bordenbestand dient bij borden van RVV-code C17-C21 een numerieke waarde te bevatten. Deze waarde geeft een hoogte, lengte, breedte of gewicht aan. Deze analyse detecteerd niet-numerieke waarden.

In [ ]:
sql='''drop table if exists int_bordenanalyse.tekstwaarde_foutief;
CREATE TABLE int_bordenanalyse.tekstwaarde_foutief AS (
    SELECT      id, rvv_code, black_code
    FROM        stg_ndw_borden.ndw_bebording_raw_catc
    WHERE       rvv_code IN ('C17','C18','C19','C20','C21') 
    AND         black_code !~ '^[-+]?[0-9]*\.?[0-9]+([eE][-+]?[0-9]+)?$' --regexp voor numerieke waarden
    ORDER BY    rvv_code, black_code
);'''
cur.execute(sql)

## Wegvak koppeling
De kolom 'location.road.wvk_id' bevat geen bruikbare waarde. Op basis van nabijheid wordt een voorstel gedaan, waar mogelijk


In [ ]:
# old
sql='''drop table if exists int_bordenanalyse.ndw_wvk_foutief;
CREATE TABLE int_bordenanalyse.ndw_wvk_foutief AS (
SELECT  id as bord_id, geom as bord_geom,
        CASE WHEN dist<10 THEN wvk_id ELSE NULL END as voorgestelde_wvk_id,
        CASE WHEN dist<10 THEN roadgeom ELSE NULL END as wvk_geom,
        CASE WHEN dist<10 THEN dist ELSE NULL END as wvk_dist
FROM (
    SELECT  *
            ,row_number() OVER (PARTITION BY sub.id ORDER BY dist ASC) AS rn
    FROM    (
        SELECT      b.id, b."location.road.name", b."location.county.name", b."location.road.wvk_id",b.geom
                    ,w.wvk_id
                    ,wd.geom as roadgeom
                    ,b.geom <-> ST_transform(wd.geom,28992) as dist
        FROM        stg_ndw_borden.ndw_bebording_raw_geom_subselection b
        LEFT JOIN   stg_ndw_wegen.nwb_raw_subselection w
        ON          b."location.road.name"=w.stt_naam
        LEFT JOIN   prd_ndw_wegen.nwb_wegvakken_directed wd
        ON          w.wvk_id=wd.id
        WHERE       b."location.road.wvk_id" IS NULL
        ORDER BY    b."location.road.name"
                    , dist
    ) sub
) sub2
WHERE       rn=1
ORDER BY    id,wvk_dist
);'''
# cur.execute(sql)

In [ ]:
sql='''drop table if exists int_bordenanalyse.ndw_wvk_foutief;
CREATE TABLE int_bordenanalyse.ndw_wvk_foutief AS (
    WITH borden_wvk_dir_null AS (
            SELECT  *
            FROM    prd_ndw_borden.ndw_bebording_verrijkt_3
            WHERE   nwb_wegvak_id_volgens_ndw_directed IS NULL
        )
        -- Wel NDW-wegvak, niet te koppelen op richting
        ,borden_windrichting AS (
            SELECT  *
            FROM    borden_wvk_dir_null
            WHERE   nwb_wegvak_id_volgens_ndw IS NOT NULL
        )
        -- Geen NDW-wegvak
        ,borden_wvk_null AS (
            SELECT  *
            FROM    borden_wvk_dir_null
            WHERE   nwb_wegvak_id_volgens_ndw IS NULL
        )
        -- Geen NDW-wegvak, wel via script
        ,borden_script AS (
            SELECT  *
            FROM    borden_wvk_null
            WHERE   _wegvak_id_volgens_script IS NOT NULL
        )
        -- Geen NDW-wegvak, geen voorstel via script
        ,borden_script_null AS (
            SELECT  *
            FROM    borden_wvk_null
            WHERE   _wegvak_id_volgens_script IS NULL
        )
        -- Wel NDW-wegvak, mismatch met script => niet rapporteren maar wel detecteren
        ,borden_script_mismatch AS (
            SELECT  *
            FROM    prd_ndw_borden.ndw_bebording_verrijkt_3
            WHERE   nwb_wegvak_id_volgens_ndw IS NOT NULL
            AND     _wegvak_id_volgens_script IS NOT NULL
            AND     nwb_wegvak_id_volgens_ndw::INT != _wegvak_id_volgens_script
        )
        -- Samen is dit de volledige set aan borden die in borden_wvk_dir_null voorkomt
        ,borden_union AS (
            SELECT  bord_id, geom
                    ,'Onbekende_rijrichting' as code
                    ,'Bord niet te koppelen aan rijrichting' as beschrijving
                    ,nwb_wegvak_id_volgens_ndw::INT as voorgestelde_wvk_id
            FROM    borden_windrichting
            UNION
            SELECT  bord_id, geom
                    ,'Onbekende_NWB_ID' as code
                    ,'Geen NWB wegvak ID beschikbaar' as beschrijving
                    ,abs(_wegvak_id_volgens_script::INT) as voorgestelde_wvk_id
            FROM    borden_script
            UNION
            SELECT  bord_id, geom
                    ,'Onbekende_NWB_ID' as code
                    ,'Geen NWB wegvak ID beschikbaar' as beschrijving
                    ,NULL::INT as voorgestelde_wvk_id
            FROM    borden_script_null
        )
    SELECT      bu.*
                ,wd.geom as wvk_geom
                ,brgs."location.county.name" as gemeentenaam
    FROM        borden_union as bu
    LEFT JOIN   prd_ndw_wegen.nwb_wegvakken_directed as wd
    ON          bu.voorgestelde_wvk_id::INT = wd.id
    LEFT JOIN   stg_ndw_borden.ndw_bebording_raw_geom_subselection as brgs
    ON          bu.bord_id = brgs.id
);'''
cur.execute(sql)

# Bijhouden van huidige, nieuwe, bekende, verholpen issues
- Huidig: detectie uit huidige bordenbestand
- Bekend: detectie uit voorgaande bordenbestanden / historie
- Nieuw: huidige issues die niet voorkomen in bekend
- Verholpen: bekende issues die niet voorkomen in huidig

In [26]:
sql='''drop table if exists int_bordenanalyse.issues_huidig;
CREATE TABLE int_bordenanalyse.issues_huidig AS (
    WITH tekstwaarde_foutief AS (
            SELECT      tf.id as bord_id
                        ,'Onbekende_getalwaarde' as code
                        ,'Onbekende getalwaarde voor beperkingsbord' as beschrijving
                        ,bv3.gemeentenaam
                        ,bv3.geom
            FROM        int_bordenanalyse.tekstwaarde_foutief as tf
            LEFT JOIN   prd_ndw_borden.ndw_bebording_verrijkt_3 as bv3
            ON          tf.id = bv3.bord_id
        )
        ,union_fouten as (
            SELECT  bord_id, code, beschrijving, gemeentenaam, geom
            FROM    tekstwaarde_foutief
            UNION
            SELECT  bord_id, code, beschrijving, gemeentenaam, geom
            FROM    int_bordenanalyse.ndw_wvk_foutief
        )
    SELECT  *
            ,now() as detectie_datum
    FROM    union_fouten
);'''
cur.execute(sql)

In [27]:
# init tabel als hij niet bestaat
sql='''CREATE TABLE IF NOT EXISTS prd_bordenanalyse.issues_bekend AS (
    SELECT  *
            ,'' as status
            ,detectie_datum as mutatie_datum
    FROM    int_bordenanalyse.issues_huidig
    LIMIT   0
);'''
cur.execute(sql)
sql_alter='''ALTER TABLE prd_bordenanalyse.issues_bekend ADD COLUMN issue_id SERIAL PRIMARY KEY;'''
cur.execute(sql_alter)

In [28]:
# nieuwe issues = subset van huidig die niet voorkomt in bekend
sql='''drop table if exists int_bordenanalyse.issues_nieuw;
CREATE TABLE int_bordenanalyse.issues_nieuw AS (
    SELECT  *
    FROM    int_bordenanalyse.issues_huidig as h
    WHERE NOT EXISTS (
        SELECT  *
        FROM    prd_bordenanalyse.issues_bekend as b
        WHERE   h.bord_id = b.bord_id AND h.code = b.code
        AND     status != 'verholpen_in_bron'
    )
);'''
cur.execute(sql)

In [29]:
# verholpen issues = subset van bekend die niet voorkomt in huidig
sql='''drop table if exists int_bordenanalyse.issues_verholpen;
CREATE TABLE int_bordenanalyse.issues_verholpen AS (
    SELECT  *
    FROM    prd_bordenanalyse.issues_bekend as b
    WHERE NOT EXISTS (
        SELECT  *
        FROM    int_bordenanalyse.issues_huidig as h
        WHERE   h.bord_id = b.bord_id AND h.code = b.code
    )
);'''
cur.execute(sql)

In [30]:
# Update bekende issues door de nieuwe issues toe te voegen en de verholpen en/of oudere issues te markeren
sql_ins='''
INSERT INTO prd_bordenanalyse.issues_bekend(bord_id,code,beschrijving,gemeentenaam,detectie_datum,mutatie_datum,status,geom)
SELECT      n.bord_id,n.code,n.beschrijving,n.gemeentenaam,n.detectie_datum
            ,detectie_datum as mutatie_datum
            ,'nieuw' as status
            ,n.geom
FROM        int_bordenanalyse.issues_nieuw as n;'''
cur.execute(sql_ins)

sql_upd='''
UPDATE      prd_bordenanalyse.issues_bekend as ib
SET         status = 'verholpen_in_bron'
            ,mutatie_datum = now()
FROM        int_bordenanalyse.issues_verholpen as iv
WHERE       ib.bord_id = iv.bord_id AND ib.code = iv.code;'''
cur.execute(sql_upd)

sql_upd2='''
UPDATE      prd_bordenanalyse.issues_bekend as ib
SET         status = 'bekend'
            ,mutatie_datum = now()
WHERE       detectie_datum < (now() - '7 day'::interval)
AND         status = 'nieuw';'''
cur.execute(sql_upd2)